In [1]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [2]:
def createsentances(data):
    games = data["games"]
    card_history = []
    for i in range(len(games)):
        g = games[i]
        if not(g.has_key("card_history")) or g["card_history"] != []:
            card_history.append(pd.DataFrame(g["card_history"]))
    sentances = []
    for i in range(len(card_history)):
        c = card_history[i]
        me = c[c.player=="me"]
        op = c[c.player=="opponent"]
        me = me["card"]
        op = op['card']
        sentances.append([i["name"] for i in me])
        sentances.append([i["name"] for i in op])
    return sentances

In [3]:
data = pd.read_json("2018-01.json");

In [4]:
sentances = createsentances(data)


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [13]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHeartStone = requests.get(urlApi).json()

In [14]:
listNameCard = [e["name"] for e in ApiHeartStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [15]:
sentancesFiltered = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [16]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'aaparition est inférieire à celui indiqué
size nombre de layers
workers parallélizer sur un nombre de coeurs

In [17]:
model = Word2Vec(sentancesFiltered,min_count=10,size=200,workers=4)

sauvegarde du model

In [18]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement
model = gensim.models.Word2Vec.load('word2vecModel')
model.train(more_sentences)

test de la  similarité

In [19]:
model.wv.similarity("Open the Waygate","Fireball")

0.4985665814165212